# [Datasets Introduction for TensorFlow v.1.3.](# https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html)  

cf. Blogpost [Introduction to TensorFlow Datasets and Estimators
Tuesday, September 12, 2017
](https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html), for the [Google Developers Blog](https://developers.googleblog.com/), https://goo.gl/Ujm2Ep  

Also the code in Python was [here](https://github.com/mhyttsten/Misc/blob/master/Blog_Estimators_DataSet.py)

In [1]:
import tensorflow
import tensorflow as tf

In [2]:
import os, sys
import urllib

In [3]:
# Check that we have the correct TensorFlow version installed  
tf_version = tf.__version__
print("TensorFlow version: {}".format(tf_version))
assert "1.3" <= tf_version, "TensorFlow r1.3 or later is needed"

TensorFlow version: 1.3.0


In [4]:
# Windows users: You only need to change PATH, 
# otherwise I have changed this PATH to be a local relative path `./` 
# from the original, PATH = "/tmp/tf_dataset_and_estimator_apis"
print(os.sep)
PATH = '.'

/


Fetch and store Training and Test dataset files 

In [5]:
PATH_DATASET = PATH + os.sep + "dataset"
print(PATH_DATASET)
FILE_TRAIN = PATH_DATASET + os.sep + "iris_training.csv"
FILE_TEST  = PATH_DATASET + os.sep + "iris_test.csv"
URL_TRAIN  = "http://download.tensorflow.org/data/iris_training.csv"
URL_TEST   = "http://download.tensorflow.org/data/iris_test.csv"

./dataset


In [6]:
def downloadDataset(url, file):
    if not os.path.exists(PATH_DATASET):
        os.makedirs(PATH_DATASET)
    if not os.path.exists(file):
        data = urllib.urlopen(url).read()
        with open(file,"w") as f:
            f.write(data)
            f.close()

In [15]:
downloadDataset(URL_TRAIN,FILE_TRAIN)
downloadDataset(URL_TEST,FILE_TEST)

In [7]:
os.listdir( PATH_DATASET )

['iris_training.csv', 'iris_test.csv']

In [17]:
# tf.logging.set_verbosity(tf.logging.INFO)  

In [8]:
# The CSV features in our training & test data
feature_names = [
    'SepalLength',
    'SepalWidth',
    'PetalLength',
    'PetalWidth'
]

## Introducing the Datasets   

cf. https://developers.googleblog.com/2017/09/introducing-tensorflow-datasets.html  

Datasets is a new way to create input pipelines to TensorFlow models.  This API is much more performant than using `feed_dict` or the queue-based pipelines, and it's cleaner and easier to use.  

Although Datasets still resides in `tf.contrib.data` at 1.3, it's expected this API will move to core at 1.4, so it's high time to take it for a test drive.  

At a high-level, Datasets consists of the following classes and relations:

$$
\text{TextLineDataset}, \text{TFRecordDataset}, \text{FixedLengthRecordDataset} \in \text{subclass} \to \text{Dataset} \xrightarrow{\text{instantiates}} \text{Iterator}
$$  

where  
* Dataset : Base class containing methods to create and transform datasets.  Also, allows you to initialize a dataset from data in memory, or from a Python generator  
* TextLineDataset : Reads lines from text files.  
* TFRecordDataset : Reads records from TFRecord files.
* FixedLengthRecordDataset : Reads fixed size records from binary files. 
* Iterator : Provdes a way to access 1 dataset element at a time 

Create an input function, reading a file using the Dataset API.  
Then provide the results to the Estimator API.  

Arguments for this function are:
* `file_path` : data file to read
* `perform_shuffle` : whether record order should be randomized 
* `repeat_count` : number of times to iterate over records in the dataset.  e.g. if `repeat_count=1`, then each record is read once.  If `repeat_count=None`, iteration will continue forever.  

In [9]:
def my_input_fn(file_path, perform_shuffle=False, repeat_count=1):
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0.],[0.,],[0.,],[0.,], [0]])
        label = parsed_line[-1:] # Last element is the label
        del parsed_line[-1] # Delete last element
        features = parsed_line # Everything but last elements are the features 
        d = dict(zip(feature_names, features)), label
        return d  
    
    dataset = (tf.contrib.data.TextLineDataset(file_path) # Read text file 
              .skip(1) # Skip header row
              .map(decode_csv)) # Transform each elem by applying decode_csv fn
    if perform_shuffle:
        # Randomizes input using a window of 256 elements (read into memory)
        dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count) # Repeats dataset this # times 
    dataset = dataset.batch(32) # Batch size to use 
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels= iterator.get_next()
    return batch_features, batch_labels

When we train our model, we'll need a function that reads the input file and returns the feature and label data.  Estimators require that you create a function that'll have a return value of a tuple.  

The return value must be a 2-element tuple organized as follows:
* The 1st element must be a `dict` in which each input feature is a key, and then a list of values for the training batch.  
* 2nd element is a list of labels for the training batch.  

Since we're returning a batch of input features and training labels, it means that all lists in the return statement will have equal lengths.  Technically speaking, whenever we referred to "list" here, we actually mean a 1-dim. TensorFlow tensor.  

Note the following: 
* `TextLineDataset` : The Dataset API will do a lot of memory management for you when you're using its file-based datasets.  You can, for example, read in dataset files much larger than memory or read in multiple files by specifying a list as argument 
* `shuffle` : reads buffer_size records, then shuffles (randomizes) their order 
* `map` : calls the `decode_csv` function with each element in the dataset as an argument (since we're using `TextLineDataset`, each element will be a line of `.csv` text).  Then we apply `decode_csv` to each of the lines  
* `decode_csv` : splits each line into fields, providing the default values if necessary.  Then returns a dict with the field keys and field values.  The map function updates each elem (line) in the dataset with the `dict`.  

That's an introduction to Datasets!  For fun, use this function to print the first batch:

In [10]:
next_batch = my_input_fn(FILE_TRAIN, True) # Will return 32 random elements 

In [11]:
# Now let's try it out, retrieving and printing 1 batch of data.
# Although this code looks strange, you don't need to understand
# the details
with tf.Session() as sess:
    first_batch = sess.run(next_batch)
print(type(first_batch)) # tuple
print(len(first_batch)) # 2
print(type(first_batch[0])) # dict
print(first_batch[0].keys())
print(type(first_batch[0]['SepalLength'] )) # np.array
print(type(first_batch[0][ feature_names[1]] )) # np.array
print(type(first_batch[0][ feature_names[2]] )) # np.array
print(type(first_batch[0][ feature_names[3]] )) # np.array
print(first_batch[0]['SepalLength'].shape ) # np.array
print(first_batch[0][ feature_names[1]].shape ) # np.array
print(first_batch[0][ feature_names[2]].shape ) # np.array
print(first_batch[0][ feature_names[3]].shape ) # np.array
print(type(first_batch[1])) # np.array
print(first_batch[1].shape) # (32,1)
print(first_batch)

<type 'tuple'>
2
<type 'dict'>
['SepalLength', 'PetalWidth', 'PetalLength', 'SepalWidth']
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
(32,)
(32,)
(32,)
(32,)
<type 'numpy.ndarray'>
(32, 1)
({'SepalLength': array([ 5.        ,  4.4000001 ,  5.        ,  6.69999981,  5.        ,
        5.        ,  4.69999981,  6.5       ,  5.        ,  6.4000001 ,
        7.69999981,  6.30000019,  4.4000001 ,  4.80000019,  5.69999981,
        6.        ,  6.4000001 ,  5.9000001 ,  6.        ,  5.        ,
        5.19999981,  6.        ,  6.0999999 ,  5.0999999 ,  6.0999999 ,
        5.        ,  7.30000019,  5.69999981,  5.5999999 ,  4.9000001 ,
        4.4000001 ,  6.5       ], dtype=float32), 'PetalWidth': array([ 1.        ,  0.2       ,  0.2       ,  2.29999995,  0.40000001,
        0.60000002,  0.2       ,  2.20000005,  0.30000001,  1.79999995,
        2.20000005,  2.4000001 ,  0.2       ,  0.1       ,  1.29999995,
        1.60000002,  1.89999998,  

Indeed, as a sanity check, we can look at the `.csv` files in `pandas`:  

In [12]:
import pandas
import pandas as pd

In [13]:
first_batch_DF = pd.read_csv(FILE_TRAIN)
print(first_batch_DF.head() )
first_batch_DF.describe()

   120    4  setosa  versicolor  virginica
0  6.4  2.8     5.6         2.2          2
1  5.0  2.3     3.3         1.0          1
2  4.9  2.5     4.5         1.7          2
3  4.9  3.1     1.5         0.1          0
4  5.7  3.8     1.7         0.3          0


,120,4,setosa,versicolor,virginica
count,120.000000,120.000000,120.000000,120.000000,120.000000
mean,5.845000,3.065000,3.739167,1.196667,1.000000
std,0.868578,0.427156,1.822100,0.782039,0.840168
min,4.400000,2.000000,1.000000,0.100000,0.000000
25%,5.075000,2.800000,1.500000,0.300000,0.000000
50%,5.800000,3.000000,4.400000,1.300000,1.000000
75%,6.425000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


## Introducing Estimators  

In [14]:
# Create the feature_columns, which specifies the input to our model
# All our input features are numeric, so use numeric_column for each one 
feature_columns = [tf.feature_column.numeric_column(k) for k in feature_names]

In [15]:
# Create a deep neural network regression classifier 
# Use the DNNClassifier pre-made estimator  
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns, # The input features to our model
    hidden_units=[10,10], # Two layers, each with 10 neurons
    n_classes=3,
    model_dir=PATH) # Path to where checkpoints etc. are stored  

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '.', '_save_summary_steps': 100}


## Training the model 

In [16]:
# Train our model, use the previous function my_input_fn
# Input to training is a file with training example
# Stop training after 8 iterations of the training of the data (epochs)
classifier.train(
    input_fn=lambda: my_input_fn(FILE_TRAIN,True,8))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model.ckpt-30
INFO:tensorflow:Saving checkpoints for 31 into ./model.ckpt.
INFO:tensorflow:loss = 10.5335, step = 31
INFO:tensorflow:Saving checkpoints for 60 into ./model.ckpt.
INFO:tensorflow:Loss for final step: 8.40039.


`lambda: my_input_fn(FILE_TRAIN, True, 8)` is where we hook up Datasets with the Estimators!  

Estimators need data to perform training, evaluation, and prediction, and it uses the `input_fn` to fetch the data.  

Estimators require an `input_fn` with no arguments, so we create a function with no arguments using `lambda`, which calls `input_fn` with the desired arguments: `file_path`, `shuffle_setting`, `repeat_count`  

In our case, we use our `my_input_fn`, passing it  
* `FILE_TRAIN`, which is the training data file.  
* `True`, which tells the Estimator to shuffle the data.
* `8`, which tells the Estimator to and repeat the dataset 8 times.  

## Evaluating Our Trained Model  

How can we evaluate how well it's performing?  Fortunately, every Estimator contains an `evaluate` method.

In [17]:
# Evaluate our model using the examples contained in FILE_TEST
# Return value will contain evaluation_metrics such as : loss & average_loss
evaluate_result = classifier.evaluate(
    input_fn=lambda: my_input_fn(FILE_TEST, False, 4))
print("Evaluation results")
for key in evaluate_result:
    print("   {}, was: {}".format(key, evaluate_result[key]))

INFO:tensorflow:Starting evaluation at 2017-09-27-14:19:45
INFO:tensorflow:Restoring parameters from ./model.ckpt-60
INFO:tensorflow:Finished evaluation at 2017-09-27-14:19:45
INFO:tensorflow:Saving dict for global step 60: accuracy = 0.866667, average_loss = 0.316627, global_step = 60, loss = 9.4988
Evaluation results
   average_loss, was: 0.316626608372
   accuracy, was: 0.866666674614
   global_step, was: 60
   loss, was: 9.49879837036


## Making Predictions Using Our Trained Model  

In [18]:
# Predict the type of some Iris flowers. 
# Let's predict the examples in FILE_TEST, repeat only once.  
predict_results = classifier.predict(
    input_fn=lambda: my_input_fn(FILE_TEST,False,1))
print("Predictions on test file")

Predictions on test file


In [19]:
for prediction in predict_results:
    # Will print the predicted class, i.e.: 0, ,1, or 2 if the prediction 
    # is Iris Sentosa, Vericolor, Virginica, respectively.  
    print(prediction["class_ids"][0])

INFO:tensorflow:Restoring parameters from ./model.ckpt-60
1
2
0
1
1
1
0
2
1
2
2
0
2
2
1
0
1
0
0
2
0
2
2
2
1
2
0
1
2
1


### Making Predictions on Data in Memory  

How could we make predictions on data residing in other sources, for example, in memory?  

In [20]:
# Let create a memory dataset for prediction.  
# We've taken the first 3 examples in FILE_TEST.  
prediction_input = [[5.9, 3.0, 4.2, 1.5], # -> 1, Iris Versicolor 
                    [6.9, 3.1, 5.4, 2.1], # -> 2, Iris Virginica
                    [5.1, 3.3, 1.7, 0.5]] # -> 0, Iris Sentosa 

def new_input_fn():
    def decode(x):
        x = tf.split(x, 4) # Need to split into our 4 features
        # When predicting, we don't need (or have) any labels
        return dict(zip(feature_names, x)) # To build a dict of them
    
    # The from_tensor_slices function will use a memory structure as input
    dataset = tf.contrib.data.Dataset.from_tensor_slices(prediction_input)
    dataset = dataset.map(decode)
    iterator = dataset.make_one_shot_iterator()
    next_feature_batch = iterator.get_next()
    return next_feature_batch, None # In prediction, we have no labels


In [21]:
# Predict all our prediction_input
predict_results = classifier.predict(input_fn=new_input_fn)

In [22]:
# Print results
print("Predictions on memory data")
for idx, prediction in enumerate(predict_results):
    type = prediction["class_ids"][0] # Get the predicted class (index)
    if type == 0:
        print("I think: {}, is Iris Sentosa".format(prediction_input[idx]))
    elif type == 1:
        print("I think: {}, is Iris Versicolor".format(prediction_input[idx]))
    else:
        print("I think: {}, is Iris Virginica".format(prediction_input[idx]))

Predictions on memory data
INFO:tensorflow:Restoring parameters from ./model.ckpt-60
I think: [5.9, 3.0, 4.2, 1.5], is Iris Versicolor
I think: [6.9, 3.1, 5.4, 2.1], is Iris Virginica
I think: [5.1, 3.3, 1.7, 0.5], is Iris Sentosa


In [ ]:
 ' '